<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_18/HW_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Применение TensorRT для оптимизации нейросетей

В этом уроке мы рассмотрим на практике оптимизацию и инференс нейронной сети с помощью библотеки TensorRT.

Изначально TensorRT существовала в виде отдельной бибилотеки, но относительно недавно её функционал был в том числе интегрирован в TensorFlow. Именно этим мы и воспользуемся в этом уроке. При такой интеграции на выходе мы всё еще получаем TensorFlow модель, просто часть её графа будут оптимизированы и вычислены (во время инференса) с помощью TensorRT.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [1]:
!pip install tensorflow==2.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564.4 MB 2.7 kB/s 
     |████████████████████████████████| 14.8 MB 55.9 MB/s 
     |████████████████████████████████| 132 kB 50.3 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=b55e05cd35d088d3e05d2ff3810307a2cfb9ad83e0ce40263ad9b8f4fffca4ab
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68713 sha256=d7ca742699320a8df19971dbe9abe61807206edf3ecdcef13ffb3e56c94c4b41
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built clang wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled ty

In [2]:
!pip install keras==2.6.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 15.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


### Загрузка библиотек

In [4]:
import numpy as np

import tensorflow as tf

In [5]:
print(tf.__version__)

2.6.0


### Импорт TensorRT
 Будем использовать TensorRT, котрый идёт внутри TensorFlow.

 В реальной среде может потребоваться произвести дополнительную установку недостающих компонентов TensorRT, но в Google Colab уже установлено всё, что нужно.


In [6]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

### Создание модели 

Создадим свёрточную нейронную сеть с большим количеством свёрточных слоёв.

Кроме того, для TensorRT оптимизации необходимо фиксировать размер входа. Делаем это с помощью метода `_set_inputs()`

In [7]:
model = tf.keras.Sequential([
     tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.MaxPool2D((2, 2)),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.MaxPool2D((2, 2)),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.MaxPool2D((2, 2)),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(10, activation='softmax'),
])

model._set_inputs(np.zeros((1, 28, 28, 1), dtype=np.float32))

### Сохранение модели 

Сохраним модель в виде `saved_model`

In [8]:
model.save('saved_model')

### Оптимизация модели с помощью TensorFlow

А теперь давайте произведем оптимизацию нашей модели с помощью TensorRT.

Сначала создадим TensorRT конвертер (`converter`) и укажем ему путь до нашей сохранённой неоптимизированной TensorFlow модели. Если нужно как-то еще сконфигурировать процесс оптимизации, можно передать дополнительные параметры в конструктор `TrtGraphConverterV2`, но мы будем использовать параметры по умолчанию.

После этого просто вызываем метод `convert()` и TensorRT применит все свои стратегии для оптимизации нашей модели. Важно запускать этот ноутбук в режиме GPU, так как TensorRT работает только с GPU.

После оптимизации можно сохранить новую модель в стандартном формате `saved_model`

In [9]:
converter = trt.TrtGraphConverterV2(input_saved_model_dir='saved_model')
converter.convert()
converter.save('saved_model_trt')

### Загрузка оптимизированной TensorRT модели

Теперь можно загрузить оптимизированную модель и произвести инференс. Во время инференса такой модели TensorFlow будет обращаться к инференс движку TensorRT.

In [10]:
model_trt = tf.keras.models.load_model('saved_model_trt')

# Warm-up
_=model_trt(np.zeros((1, 28, 28, 1), dtype=np.float32))

### Сравнение скорости работы двух моделей

Запустим инференс для обеих моделей со случайным входом и узнаем среднюю скорость работы каждой из моделей с помощью магической команды `%%timeit`

In [11]:
%%timeit -n 10 -r 10

q = model(np.zeros((1, 28, 28, 1), dtype=np.float32))

10.8 ms ± 2.16 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [12]:
%%timeit -n 10 -r 10

q = model_trt(np.zeros((1, 28, 28, 1), dtype=np.float32))

2.21 ms ± 374 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


TensorRT модель получилась быстрее.

**[Задание 1]** Сравните скорость работы двух моделей при различных гиперпараметрах (размер батча, размер входа, количество слоёв). Рассчитайте коэффициент ускорения (во сколько раз одна модель быстрее другой) для каждой конфигурации. Постройте соответствующие графики. (Например, график зависимости ускорения от размера входа при условии, что всё остальное фиксировано). Для этого задания вам понадобится самостоятельно реализовать способ измерения среднего времени инференса модели.

In [14]:
%%timeit -n 10 -r 10

q = model(np.zeros((128, 28, 28, 1), dtype=np.float32))

The slowest run took 4.21 times longer than the fastest. This could mean that an intermediate result is being cached.
14.6 ms ± 10.1 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [15]:
%%timeit -n 10 -r 10

q = model_trt(np.zeros((128, 28, 28, 1), dtype=np.float32))

The slowest run took 6.45 times longer than the fastest. This could mean that an intermediate result is being cached.
7.95 ms ± 8.17 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


### **HOMEWORK model optimization TensorRT**

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [22]:
!ls gdrive/MyDrive/Skillbox/model_1/best_checkpoint.hdf5

best_checkpoint.hdf5


In [23]:
tf_model = tf.keras.models.load_model('gdrive/MyDrive/Skillbox/model_1/best_checkpoint.hdf5')

# Check its architecture
tf_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_19 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_20 (Dense)             (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [41]:
tf_model.save('gdrive/MyDrive/Skillbox/model_1/smodel')

In [39]:
%%timeit -n 10 -r 10
q = tf_model(np.zeros((128,784), dtype=np.float32))

1.59 ms ± 152 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [42]:
converter_new = trt.TrtGraphConverterV2(input_saved_model_dir='gdrive/MyDrive/Skillbox/model_1/smodel')
converter_new.convert()
converter_new.save('gdrive/MyDrive/Skillbox/model_1/saved_model_trt_new')

In [44]:
trt_model = tf.keras.models.load_model('gdrive/MyDrive/Skillbox/model_1/saved_model_trt_new')

# Warm-up
#_=trt_model(np.zeros((1, 28, 28, 1), dtype=np.float32))

In [50]:
%%timeit -n 10 -r 10
q = tf_model(np.zeros((10,784), dtype=np.float32))

1.36 ms ± 254 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [49]:
%%timeit -n 10 -r 10
q = trt_model(np.zeros((10,784), dtype=np.float32))

The slowest run took 5.45 times longer than the fastest. This could mean that an intermediate result is being cached.
1.26 ms ± 957 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
